In [6]:
# load dependencies
import kaggle
import zipfile
import os
import pandas as pd
import numpy as np

In [7]:
# download `.zip` from Kaggle
!kaggle datasets download -d sudalairajkumar/cryptocurrencypricehistory

cryptocurrencypricehistory.zip: Skipping, found more recently modified local copy (use --force to force download)


In [27]:
# extract all `.csv` files to a dedicated directory
with zipfile.ZipFile("cryptocurrencypricehistory.zip", 'r') as zip_ref:
    zip_ref.extractall("raw_data")

# instantiate a dictionary to hold our dataframes
frame_dict = {}

# itereate over files 
for csv in os.listdir("/Users/davidlevy/Desktop/project_1/raw_data"):
    
    # ignore hidden files
    if csv[0] != ".":
        filepath = os.path.join("raw_data", csv)
        frame_dict[csv.replace(".csv", "_df")] = pd.read_csv("raw_data/" + csv)

In [ ]:
frame_dict[0].head()